In [76]:
with open("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/NLP_LLM/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of charecters in the text: ", len(raw_text))
print("First 100 charecters of raw_text:\n", raw_text[:99])

Total number of charecters in the text:  20479
First 100 charecters of raw_text:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [77]:
# The goal is to tokenize the whole 20479 tokens of short story and special charecters into embeddings for pretraining LLM

In [78]:
import re

In [79]:
test_text = "Hello, World. This, is a test."

result = re.split(r"(\s)", test_text)

print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [80]:
result2 = re.split(r"[,.] | \s", test_text)
print(result2)

['Hello', 'World', 'This', 'is a test.']


In [81]:
result3 = re.split(r'([,.]|\s)', test_text)
print(result3)

['Hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [82]:
result = result3

In [83]:
result = [item for item in result if item.split()]

print(result)

['Hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


In [84]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(result)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'Is', ' ', 'this', '--', '', ' ', 'a', ' ', 'test', '?', '']
['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [85]:
# Now that we have our basic tokenizer ready, lets apply it to our raw_text

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [86]:
print(len(preprocessed))

4690


In [87]:
# Converting tokens into token ids

all_words = sorted(set(preprocessed))

vocab_size = len(all_words)

print(vocab_size)

1130


In [88]:
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, tup in enumerate(vocab.items()):
    print(tup)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [89]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?!_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        idl = [self.int_to_str[i] for i in ids]
        text = " ".join(idl)
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [90]:
tokenizer = SimpleTokenizerV1(vocab)

In [91]:
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

print("Text: \n", text)

ids = tokenizer.encode(text)
print("Token_ids: \n", ids)

Text: 
 "It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.
Token_ids: 
 [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [92]:
text_conv = tokenizer.decode(ids)
print(text_conv)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [93]:
# Adding special tokens

all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftoken|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}
print("Vocab size with special tokens: ", len(vocab))

Vocab size with special tokens:  1132


In [94]:
for i, token in enumerate(list(vocab.items())[-5:]):
    print(token)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftoken|>', 1130)
('<|unk|>', 1131)


In [95]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer:token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?!_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[i] for i in preprocessed]
        return ids
    
    def decode(self, ids):
        idl = [self.int_to_str[i] for i in ids]
        text = " ".join(idl)
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [96]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftoken|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftoken|> In the sunlit terraces of the palace.


In [98]:
tokenizer.encode(text)


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [99]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftoken|> In the sunlit terraces of the <|unk|>.'